In [6]:
# %%file liepin.py

# %%file liepin.py
# Code to scrape job information from the website goes here

import requests
from bs4 import BeautifulSoup

# Send request to the website
url = "https://www.liepin.com/zhaopin/?inputFrom=www_index&workYearCode=0&key=%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86%E5%B7%A5%E7%A8%8B%E5%B8%88&scene=input&ckId=3b5ozfzgjg4lllej2tlhw59ck4r860p8&dq="
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-CN,zh;q=0.9",
    "Cookie": "__gc_id=f22ffae0bcc3463da4baaebd39565220; __uuid=1691135429066.26; _ga=GA1.1.1902532824.1691135430; __tlog=1691470573010.41%7C00000000%7C00000000%7C00000000%7C00000000; XSRF-TOKEN=bFLGHjv6TSO1FP0pFVQcxQ; Hm_lvt_a2647413544f5a04f00da7eee0d5e200=1691212344,1691294573,1691458626,1691470573; acw_tc=2760829a16914742554304576e46e7ebfa068095afda8b4d9c185c66cd426e; __session_seq=4; __uv_seq=12; Hm_lpvt_a2647413544f5a04f00da7eee0d5e200=1691474255; _ga_54YTJKWN86=GS1.1.1691474240.11.1.1691474255.0.0.0",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
    # sec-ch-ua: "Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"
}
response = requests.get(url, headers=headers)
print(response.content)


b'<!DOCTYPE html>\n<html lang="en">\n<head>\n  \n    <!-- Google tag (gtag.js) -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=G-54YTJKWN86"></script>\n    <script>\n        window.dataLayer = window.dataLayer || [];\n        function gtag(){dataLayer.push(arguments);}\n        gtag(\'js\', new Date());\n\n        gtag(\'config\', \'G-54YTJKWN86\');\n    </script>\n\n  \n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n    <title>\xe3\x80\x90\xe6\x8b\x9b\xe8\x81\x98\xe4\xbf\xa1\xe6\x81\xaf_\xe4\xba\xba\xe6\x89\x8d\xe7\xbd\x91\xe6\x8b\x9b\xe8\x81\x98\xe4\xbf\xa1\xe6\x81\xaf\xe3\x80\x91-\xe7\x8c\x8e\xe8\x81\x98</title>\n    <meta name="keywords" content="\xe6\x8b\x9b\xe8\x81\x98\xe4\xbf\xa1\xe6\x81\xaf,\xe4\xba\xba\xe6\x89\x8d\xe4\xbf\xa1\xe6\x81\xaf,\xe4\xba\xba\xe6\x89\x8d\xe7\xbd\x91\xe6\x8b\x9b\xe8\x81\x98\xe4\xbf\xa1\xe6\x81\xaf" />\n    <meta name="description" content="\xe7\x8c\x8e\xe8\x81\x98\xe6\x8b\x9b\xe8\x81\x98\xe4\xbf\xa1\xe6

In [7]:
# Authenticate if necessary
# ...

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")
print(soup)

<!DOCTYPE html>

<html lang="en">
<head>
<!-- Google tag (gtag.js) -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-54YTJKWN86"></script>
<script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());

        gtag('config', 'G-54YTJKWN86');
    </script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>【招聘信息_人才网招聘信息】-猎聘</title>
<meta content="招聘信息,人才信息,人才网招聘信息" name="keywords"/>
<meta content="猎聘招聘信息网提供真实的人才网招聘信息,每日上万高薪职位招聘信息发布,是专业人才、精英人才获取招聘信息的专业平台。找2023招聘信息,就来人才网招聘信息频道。" name="description"/>
<meta content="pc" name="applicable-device"/>
<meta content="no-transform" http-equiv="Cache-Control">
<meta content="no-siteapp" http-equiv="Cache-Control"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="//concat.lietou-static.com" rel="dns-prefetch"/>
<link href="//image0.li

In [ ]:

# Extract job information
job_list = []
# ...

# Example code to print the scraped information
for job in job_list:
    print(job.title)
    print(job.company)
    print(job.location)
    print(job.salary)
    print(job.description)
    print()



